# **Simple binary classifier** 

This notebook will generate a binary classifier that will identify if an attack is benign or not. The following algorithms will be used:
* Logistic regression 
* Random forest 
* CatBoost

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import scikitplot as skplt
import eli5
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_recall_curve, average_precision_score, roc_auc_score
from catboost import CatBoostClassifier
from catboost import Pool
from eli5.sklearn import PermutationImportance
from scipy.stats import spearmanr
from scipy.cluster import hierarchy 
from scipy.spatial.distance import squareform
from collections import defaultdict
from scipy.stats import ks_2samp
from scipy.stats import describe
from imblearn.over_sampling import SMOTENC
from collections import Counter
from IPython.display import display

In [2]:
data_path = os.path.dirname(os.path.abspath('')) + "/data/combined/"
df = pd.read_pickle(data_path+'combined_cleaned.pickle')

In [3]:
X = df.drop(columns=['label', 'detect_type', 'detect_threat'])
y = df[['detect_type', 'detect_threat', 'label']]
stats = X.describe()

In [4]:
print(X.shape)
counts = X.nunique()
to_del = [i for i,v in enumerate(counts) if v == 1]
print(X.columns[to_del])
print(to_del)
X = X.drop(X.columns[to_del], axis = 1)
print(X.shape)

(16137183, 79)
Index(['bwd_psh_flags', 'bwd_urg_flags', 'fwd_byts_b_avg', 'fwd_pkts_b_avg',
       'fwd_blk_rate_avg', 'bwd_byts_b_avg', 'bwd_pkts_b_avg',
       'bwd_blk_rate_avg'],
      dtype='object')
[33, 35, 57, 58, 59, 60, 61, 62]
(16137183, 71)


In [5]:
X = X.drop(columns=['timestamp', 'dst_port'])

In [6]:
# Min Max Scaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV

from sklearn.preprocessing import StandardScaler




In [7]:
# kmeans = KMeans()
# ssd = []
# K = range(1, 30)

# for k in K:
#     kmeans = KMeans(n_clusters=k).fit(scaled_sub)
#     ssd.append(kmeans.inertia_)

# ssd

# plt.plot(K, ssd, "bx-")
# plt.xlabel("Distance Residual Sums for K Values (WCSS)")
# plt.title("Elbow Method for Optimum Number of Clusters")
# plt.show()

# kmeans = KMeans()
# visu = KElbowVisualizer(kmeans, k=(2, 20))
# visu.fit(X)
# visu.show()

In [8]:
# kmeans = KMeans(n_clusters=6).fit(scaled_sub)
# clusters = kmeans.labels_

# pd.DataFrame({"Customers": X.index, "Clusters": clusters})
# X["cluster_no"] = clusters
# X.head()

In [9]:
# X["cluster_no"] = X["cluster_no"] + 1
# X.groupby("cluster_no").agg({"cluster_no": "count"})

In [10]:
# X.groupby("cluster_no").agg(np.mean)

In [11]:
# # Plot the histogram of various clusters
# for i in X.columns:
#   plt.figure(figsize = (35, 5))
#   for j in range(1,8):
#     plt.subplot(1,8,j+1)
#     cluster = X[X['cluster_no'] == j]
#     cluster[i].hist(bins = 20)
#     plt.title('{}    \nCluster {} '.format(i,j))
  
#   plt.show()

In [12]:
# # Average Linkage Method
# hc_average = linkage(, "average")

# plt.figure(figsize=(20, 10))
# plt.title("Hierarchical Clustering")
# plt.xlabel("Observations")
# plt.ylabel("Distance")
# dendrogram(hc_average,
#            leaf_font_size=10, 
#            p=10,
#            show_contracted=True,
#           truncate_mode='level')
# plt.show()

In [13]:
plt.figure(figsize=(15,10))
correlations = spearmanr(X).correlation 
# correlations = X.corr() 


plt.figure(figsize=(25, 20))
# sns.heatmap(correlations, xticklabels=correlations.columns, yticklabels=correlations.columns, linewidths=.6, cmap= 'coolwarm', linewidth=1, linecolor='black')
sns.heatmap(correlations, xticklabels=X.columns, yticklabels=X.columns, linewidths=.6, cmap= 'coolwarm', linewidth=1, linecolor='black')

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Liberation Sans, Bitstream Vera Sans, sans-serif


<AxesSubplot:>

<Figure size 1080x720 with 0 Axes>

In [14]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform
# corr_linkage = hierarchy.ward(corr)
plt.figure(figsize=(12,5))
dissimilarity = 1 - abs(correlations)

# Z = hierarchy.linkage(correlations, 'complete')
# Z = hierarchy.linkage(correlations, 'single')
Z = hierarchy.ward(correlations)

dendro = hierarchy.dendrogram(Z, labels=X.columns, orientation='top', 
           leaf_rotation=90)

    

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Liberation Sans, Bitstream Vera Sans, sans-serif


In [15]:
# Clusterize the data
threshold = 0.8
labels = hierarchy.fcluster(Z, threshold, criterion='distance')

print(labels)
cluster_id_to_feature_ids = defaultdict(list)

for idx, cluster_id in enumerate(labels):
    cluster_id_to_feature_ids[cluster_id].append(idx)
selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]

selected_features = X.columns[selected_features].tolist()

print('Selected features:')
pd.Series(selected_features)


[ 1 16 26 22 23 24 23  1 20 30 24  1 21 30  4  9 16 27 16 10 15 15 27 15
 19 28 28 28 28 31  5  6 26 22  9  2  1 24 21 24 24  7  5 25 29 11  8  6
 25  3 21 20 21 26 23 22 24 17 29 28 18 13 14 13 13 12 14 12 12]
Selected features:


0              protocol
1         flow_duration
2          tot_fwd_pkts
3          tot_bwd_pkts
4       totlen_fwd_pkts
5       totlen_bwd_pkts
6      fwd_pkt_len_mean
7       fwd_pkt_len_std
8      bwd_pkt_len_mean
9           flow_byts_s
10          flow_pkts_s
11         flow_iat_std
12         flow_iat_min
13          fwd_iat_tot
14          fwd_iat_min
15          bwd_iat_tot
16          bwd_iat_min
17        fwd_psh_flags
18        fwd_urg_flags
19           bwd_pkts_s
20         fin_flag_cnt
21         rst_flag_cnt
22         psh_flag_cnt
23         ack_flag_cnt
24         urg_flag_cnt
25        down_up_ratio
26    init_fwd_win_byts
27     fwd_seg_size_min
28          active_mean
29           active_std
30            idle_mean
dtype: object

In [16]:

X = X[selected_features]
print(f"Remaining amount of features: {len(X.columns)}")


Remaining amount of features: 31


In [17]:
plt.figure(figsize=(15,10))
correlations = spearmanr(X).correlation
plt.figure(figsize=(25, 20))
sns.heatmap(correlations, xticklabels=X.columns, yticklabels=X.columns, linewidths=.6, cmap= 'coolwarm', linewidth=1, linecolor='black')

<AxesSubplot:>

<Figure size 1080x720 with 0 Axes>

In [18]:
# plt.figure(figsize=(15,10))

# for idx, t in enumerate(np.arange(0.2,1.1,0.1)):
    
#     # Subplot idx + 1
#     plt.subplot(3, 3, idx+1)
#     Z = hierarchy.ward(correlations)
#     # Calculate the cluster
#     labels = hierarchy.fcluster(Z, threshold, criterion='distance')
#     # Keep the indices to sort labels
#     labels_order = np.argsort(labels)

#     # Build a new dataframe with the sorted columns
#     for idx, i in enumerate(X.columns[labels_order]):
#         if idx == 0:
#             clustered = pd.DataFrame(X[i])
#         else:
#             df_to_append = pd.DataFrame(X[i])
#             clustered = pd.concat([clustered, df_to_append], axis=1)
            
#     # Plot the correlation heatmap
#     correlations = clustered.corr()
#     sns.heatmap(round(correlations,2), cmap='RdBu', vmin=-1, vmax=1, 
#                 xticklabels=False, yticklabels=False)
#     plt.title("Threshold = {}".format(round(t,2)))

In [7]:
def print_report(ds_type, cls, X_vals, y_true, y_predict, plot_pr=False, plot_roc=False):
    print(f"Classification Report ({ds_type}):")
    print(classification_report(y_true, y_predict))
    print(f"Avg Precision Score: {average_precision_score(y_true, y_predict, average='weighted')}")
    
    if plot_roc:
        print(f"ROC AUC Score: {roc_auc_score(y_true, y_predict)}")
        skplt.metrics.plot_roc(y_true, cls.predict_proba(X_vals))
        plt.show()
        
    if plot_pr:
        
        skplt.metrics.plot_precision_recall(y_true, cls.predict_proba(X_vals))
        plt.show()
        
    print('\n')

In [6]:
X_train, X_hold, y_train, y_hold = train_test_split(X, y, test_size=0.3, stratify=y.detect_type)
X_eval, X_test, y_eval, y_test = train_test_split(X_hold, y_hold, test_size=0.6, stratify=y_hold.detect_type)

X_train_oh = pd.get_dummies(X_train, columns=['protocol'])
X_eval_oh = pd.get_dummies(X_eval, columns=['protocol'])
X_test_oh = pd.get_dummies(X_test, columns=['protocol'])

In [21]:

y.label.value_counts()

Benign                      13390249
DDOS attack-HOIC              686012
DDoS attacks-LOIC-HTTP        576191
DoS attacks-Hulk              461912
Bot                           286191
FTP-BruteForce                193354
SSH-Bruteforce                187589
Infilteration                 160639
DoS attacks-SlowHTTPTest      139890
DoS attacks-GoldenEye          41508
DoS attacks-Slowloris          10990
DDOS attack-LOIC-UDP            1730
Brute Force -Web                 611
Brute Force -XSS                 230
SQL Injection                     87
Name: label, dtype: int64

In [22]:
y_train.label.value_counts()

Benign                      9373174
DDOS attack-HOIC             480208
DDoS attacks-LOIC-HTTP       403334
DoS attacks-Hulk             323338
Bot                          200334
FTP-BruteForce               135348
SSH-Bruteforce               131312
Infilteration                112447
DoS attacks-SlowHTTPTest      97923
DoS attacks-GoldenEye         29056
DoS attacks-Slowloris          7693
DDOS attack-LOIC-UDP           1211
Brute Force -Web                428
Brute Force -XSS                161
SQL Injection                    61
Name: label, dtype: int64

In [23]:
y_eval.label.value_counts()

Benign                      1606830
DDOS attack-HOIC              82322
DDoS attacks-LOIC-HTTP        69143
DoS attacks-Hulk              55429
Bot                           34343
FTP-BruteForce                23202
SSH-Bruteforce                22511
Infilteration                 19277
DoS attacks-SlowHTTPTest      16787
DoS attacks-GoldenEye          4981
DoS attacks-Slowloris          1319
DDOS attack-LOIC-UDP            207
Brute Force -Web                 73
Brute Force -XSS                 28
SQL Injection                    10
Name: label, dtype: int64

In [24]:
y_test.label.value_counts()

Benign                      2410245
DDOS attack-HOIC             123482
DDoS attacks-LOIC-HTTP       103714
DoS attacks-Hulk              83145
Bot                           51514
FTP-BruteForce                34804
SSH-Bruteforce                33766
Infilteration                 28915
DoS attacks-SlowHTTPTest      25180
DoS attacks-GoldenEye          7471
DoS attacks-Slowloris          1978
DDOS attack-LOIC-UDP            312
Brute Force -Web                110
Brute Force -XSS                 41
SQL Injection                    16
Name: label, dtype: int64

In [27]:
percentage_benign = len(y_train.detect_threat[y_train.detect_threat == 0]) / len(y_train.detect_threat)

print('Class occurences:')
print(y_train.detect_threat.value_counts())
print('Percentage of benign samples: %.4f' % percentage_benign)

Class occurences:
0    9373174
1    1922854
Name: detect_threat, dtype: int64
Percentage of benign samples: 0.8298


In [28]:
cls_dummy = DummyClassifier('most_frequent')
cls_dummy.fit(X_train, y_train.detect_threat)

print_report('Train', cls_dummy, X_train, y_train.detect_threat, cls_dummy.predict(X_train))
print_report('Eval', cls_dummy, X_eval, y_eval.detect_threat, cls_dummy.predict(X_eval))

/home/am520/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass strategy=most_frequent as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Classification Report (Train):


/home/am520/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/am520/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/am520/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.83      1.00      0.91   9373174
           1       0.00      0.00      0.00   1922854

    accuracy                           0.83  11296028
   macro avg       0.41      0.50      0.45  11296028
weighted avg       0.69      0.83      0.75  11296028

Avg Precision Score: 0.1702239052523595


Classification Report (Eval):


/home/am520/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/am520/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.83      1.00      0.91   1606830
           1       0.00      0.00      0.00    329632

    accuracy                           0.83   1936462
   macro avg       0.41      0.50      0.45   1936462
weighted avg       0.69      0.83      0.75   1936462

Avg Precision Score: 0.17022384121144643




/home/am520/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
scaler = StandardScaler()
scaler.fit(X_train_oh)

StandardScaler()

In [9]:
cls_lr = LogisticRegression(solver='saga', n_jobs=-1, verbose=2)
cls_lr.fit(scaler.transform(X_train_oh), y_train.detect_threat)

print_report('Train', cls_lr, scaler.transform(X_train_oh), y_train.detect_threat, cls_lr.predict(scaler.transform(X_train_oh)))
print_report('Eval', cls_lr, scaler.transform(X_eval_oh), y_eval.detect_threat, cls_lr.predict(scaler.transform(X_eval_oh)), plot_pr=True)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


Epoch 1, change: 1.00000000
Epoch 2, change: 0.28278394


KeyboardInterrupt: 

Epoch 3, change: 0.17255325
Epoch 4, change: 0.12904531
Epoch 5, change: 0.10391641
Epoch 6, change: 0.08701572
Epoch 7, change: 0.07455018
Epoch 8, change: 0.06491609
Epoch 9, change: 0.05722247
Epoch 10, change: 0.05094921
Epoch 11, change: 0.04576043
Epoch 12, change: 0.04140255
Epoch 13, change: 0.03768921
Epoch 14, change: 0.03449060
Epoch 15, change: 0.03171807
Epoch 16, change: 0.02929915
Epoch 17, change: 0.02716333
Epoch 18, change: 0.02527434


In [31]:
cls_forest = RandomForestClassifier(verbose=1, n_jobs=-1, class_weight='balanced')
cls_forest.fit(X_train_oh, y_train.detect_threat)

print_report('Train', cls_forest, X_train_oh, y_train.detect_threat, cls_forest.predict(X_train_oh))
print_report('Eval', cls_forest, X_eval_oh, y_eval.detect_threat, cls_forest.predict(X_eval_oh), plot_pr=True)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 19.6min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   58.7s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  2.2min finished


Classification Report (Train):
              precision    recall  f1-score   support

           0       1.00      0.99      1.00   9373174
           1       0.97      0.98      0.98   1922854

    accuracy                           0.99  11296028
   macro avg       0.98      0.99      0.99  11296028
weighted avg       0.99      0.99      0.99  11296028

Avg Precision Score: 0.956345023118038




[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    9.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   21.6s finished


Classification Report (Eval):
              precision    recall  f1-score   support

           0       0.99      0.99      0.99   1606830
           1       0.96      0.95      0.96    329632

    accuracy                           0.99   1936462
   macro avg       0.98      0.97      0.97   1936462
weighted avg       0.99      0.99      0.99   1936462

Avg Precision Score: 0.9263833835947416


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    9.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   21.3s finished


In [7]:
def log_reg(x, y):
    scaledd = StandardScaler()
    scaledd.fit(x)   
    log_reg = LogisticRegression(solver='saga', n_jobs=-1, verbose=2)
    return log_reg.fit(scaledd.transform(x), y)

log_reg = log_reg(X_train_oh, y_train.detect_threat)


In [2]:
train_pool = Pool(X_train, y_train.detect_threat, cat_features=['protocol'])
eval_pool = Pool(X_eval, y_eval.detect_threat, cat_features=['protocol'])
test_pool = Pool(X_test, cat_features=['protocol'])

NameError: name 'X_train' is not defined

In [33]:

    
cls_cb = CatBoostClassifier(loss_function='Logloss',
                            eval_metric='Recall',                        
                            task_type='GPU',
                            verbose=True)

cls_cb.fit(train_pool, eval_set=eval_pool)

Learning rate set to 0.032213
0:	learn: 0.9129097	test: 0.9124175	best: 0.9124175 (0)	total: 667ms	remaining: 11m 5s
1:	learn: 0.9129097	test: 0.9124175	best: 0.9124175 (0)	total: 1.32s	remaining: 10m 56s
2:	learn: 0.9129097	test: 0.9124175	best: 0.9124175 (0)	total: 1.89s	remaining: 10m 26s
3:	learn: 0.9129097	test: 0.9124175	best: 0.9124175 (0)	total: 2.59s	remaining: 10m 44s
4:	learn: 0.7937561	test: 0.7941553	best: 0.9124175 (0)	total: 3.2s	remaining: 10m 37s
5:	learn: 0.7937561	test: 0.7941553	best: 0.9124175 (0)	total: 3.77s	remaining: 10m 24s
6:	learn: 0.7937561	test: 0.7941553	best: 0.9124175 (0)	total: 4.41s	remaining: 10m 26s
7:	learn: 0.7937561	test: 0.7941553	best: 0.9124175 (0)	total: 5.03s	remaining: 10m 23s
8:	learn: 0.7937561	test: 0.7941553	best: 0.9124175 (0)	total: 5.64s	remaining: 10m 21s
9:	learn: 0.7937561	test: 0.7941553	best: 0.9124175 (0)	total: 6.22s	remaining: 10m 15s
10:	learn: 0.7937561	test: 0.7941553	best: 0.9124175 (0)	total: 6.74s	remaining: 10m 5s
11:	

In [34]:
print_report('Train', cls_cb, train_pool, y_train.detect_threat, cls_cb.predict(train_pool))
print_report('Eval', cls_cb, eval_pool, y_eval.detect_threat, cls_cb.predict(eval_pool), plot_pr=True)

Classification Report (Train):
              precision    recall  f1-score   support

           0       0.99      0.99      0.99   9373174
           1       0.96      0.95      0.96   1922854

    accuracy                           0.99  11296028
   macro avg       0.98      0.97      0.97  11296028
weighted avg       0.99      0.99      0.99  11296028

Avg Precision Score: 0.9237081867638586


Classification Report (Eval):
              precision    recall  f1-score   support

           0       0.99      0.99      0.99   1606830
           1       0.96      0.95      0.96    329632

    accuracy                           0.99   1936462
   macro avg       0.98      0.97      0.97   1936462
weighted avg       0.99      0.99      0.99   1936462

Avg Precision Score: 0.9239507000795788


In [35]:
estimator = cls_cb

In [36]:

print_report('Test', estimator, test_pool, y_test.detect_threat, estimator.predict(test_pool), plot_pr=True)

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.99      0.99      0.99   2410245
           1       0.96      0.95      0.96    494448

    accuracy                           0.99   2904693
   macro avg       0.98      0.97      0.97   2904693
weighted avg       0.99      0.99      0.99   2904693

Avg Precision Score: 0.9239680238722019


In [2]:
def calculate_misclassifiations(y, predictions):
    classifications = y.copy()
    classifications['pred'] = predictions
    counts = classifications.label.value_counts()
    
    misclassifications = classifications[classifications.detect_threat != classifications.pred]
    mc = pd.DataFrame(misclassifications.label.value_counts())
    mc['percentage'] = mc.apply(lambda x: x[0] / counts[x.name], axis=1)
    mc = mc.sort_values('percentage', ascending=False)
    
    print('Misclassifications:')
    display(mc)

In [38]:
calculate_misclassifiations(y_test, estimator.predict(test_pool))



Misclassifications:


,label,percentage
SQL Injection,13,0.812500
Infilteration,22722,0.785821
Brute Force -Web,81,0.736364
Brute Force -XSS,16,0.390244
DoS attacks-Slowloris,21,0.010617
Benign,19190,0.007962
Bot,155,0.003009
DDoS attacks-LOIC-HTTP,62,0.000598
DDOS attack-HOIC,14,0.000113
DDOS attack-LOIC-UDP,0,0.000000


In [2]:
def get_data():
    data_path = os.path.dirname(os.path.abspath('')) + "/data/combined/"
    df = pd.read_pickle(data_path+'combined_cleaned.pickle')
    return df

def del_std_equal_to_zero(d):
    print(d.shape)
    counts = d.nunique()
    to_del = [i for i,v in enumerate(counts) if v == 1]
    print(d.columns[to_del])
    print(to_del)
    d = d.drop(d.columns[to_del], axis = 1)
    print(d.shape)
    return d

def correlations(d):
    # plt.figure(figsize=(15,10))
    # placing the deature dataset into a spearmanr structure and then into a correlation
    correlations = spearmanr(d).correlation 
    # plt.figure(figsize=(25, 20))
    # before picture of the correlations 
    # sns.heatmap(correlations, xticklabels=d.columns, yticklabels=d.columns, linewidths=.6, cmap= 'coolwarm', linewidth=1, linecolor='black')
    plt.figure(figsize=(12,5))
    # Z = hierarchy.linkage(correlations, 'complete')
    # Z = hierarchy.linkage(correlations, 'single')
    # plt.figure(figsize=(15,10))
    Z = hierarchy.ward(correlations)
    dendro = hierarchy.dendrogram(Z, labels=d.columns, orientation='top', leaf_rotation=90)
    # # Clusterize the data
    threshold = 0.95
    labels = hierarchy.fcluster(Z, threshold, criterion='distance')
    cluster_id_to_feature_ids = defaultdict(list)
    for idx, cluster_id in enumerate(labels):
        cluster_id_to_feature_ids[cluster_id].append(idx)
    selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]
    selected_features = d.columns[selected_features].tolist()
    pd.Series(selected_features)
    d = d[selected_features]
    print("Remaining amount of features:", len(d.columns))
    ### Plot the correlation heatmap of the selkected columns
    # plt.figure(figsize=(15,10))
    # correlations = spearmanr(d).correlation
    # # plt.figure(figsize=(25, 20))
    # sns.heatmap(correlations, xticklabels=d.columns, yticklabels=d.columns, linewidths=.6, cmap= 'coolwarm', linewidth=1, linecolor='black')
    return d

def print_report(ds_type, cls, X_vals, y_true, y_predict, plot_pr=False, plot_roc=False):
    print(f"Classification Report ({ds_type}):")
    print(classification_report(y_true, y_predict))
    print(f"Avg Precision Score: {average_precision_score(y_true, y_predict, average='weighted')}")
    
    if plot_roc:
        print(f"ROC AUC Score: {roc_auc_score(y_true, y_predict)}")
        skplt.metrics.plot_roc(y_true, cls.predict_proba(X_vals))
        plt.show()
        
    if plot_pr:
        skplt.metrics.plot_precision_recall(y_true, cls.predict_proba(X_vals))
        plt.show()
        
    print('\n')

def split_data(X, y, size, label):
    return train_test_split(X, y, test_size=size, stratify=label)

def one_hot_encoding(x, col):
    df = pd.get_dummies(x, columns=col)
    return df

def dummy_classifier(x,y):
    cls_dum = DummyClassifier('most_frequent')
    return cls_dum.fit(x, y)

def scale_data(x):
    scaledd = StandardScaler()
    return scaledd.fit(x)

def log_reg(x, y, sd):
    log_reg = LogisticRegression(solver='saga', n_jobs=-1, verbose=2)
    return log_reg.fit(sd.transform(x), y)

def random_f(x,y):
    rf = RandomForestClassifier(verbose=1, n_jobs=-1, class_weight='balanced')
    return rf.fit(x, y)

def calculate_misclassifiations(y, predictions):
    classifications = y.copy()
    classifications['pred'] = predictions
    counts = classifications.label.value_counts()
    
    misclassifications = classifications[classifications.detect_threat != classifications.pred]
    mc = pd.DataFrame(misclassifications.label.value_counts())
    mc['percentage'] = mc.apply(lambda x: x[0] / counts[x.name], axis=1)
    mc = mc.sort_values('percentage', ascending=False)
    
    print('Misclassifications:')
    display(mc)

df = get_data() 

X = df.drop(columns=['label', 'detect_type', 'detect_threat'])
y = df[['label', 'detect_type', 'detect_threat']]
X = del_std_equal_to_zero(X)
X = X.drop(columns=['timestamp', 'dst_port'])
X = correlations(X)



X_train, X_hold, y_train, y_hold = split_data(X, y, 0.3, y.detect_type)
X_eval, X_test, y_eval, y_test = split_data(X_hold, y_hold, 0.5, y_hold.detect_type)

X_train_oh = one_hot_encoding(X_train, ['protocol'])
X_eval_oh = one_hot_encoding(X_eval, ['protocol'])
X_test_oh = one_hot_encoding(X_test, ['protocol'])

# X_train, X_hold, y_train, y_hold = train_test_split(X, y, test_size=0.3, stratify=y.detect_type)
# X_eval, X_test, y_eval, y_test = train_test_split(X_hold, y_hold, test_size=0.6, stratify=y_hold.detect_type)

# X_train_oh = pd.get_dummies(X_train, columns=['protocol'])
# X_eval_oh = pd.get_dummies(X_eval, columns=['protocol'])
# X_test_oh = pd.get_dummies(X_test, columns=['protocol'])

benchmark_dummy = dummy_classifier(X_train, y_train.detect_threat)

print_report('Train', benchmark_dummy, X_train, y_train.detect_threat, benchmark_dummy.predict(X_train)) 
print_report('Eval', benchmark_dummy, X_eval, y_eval.detect_threat, benchmark_dummy.predict(X_eval))

scaled_data = scale_data(X_train_oh)
# scaled_data = StandardScaler()
# scaled_data.fit(X_train_oh)
log_reg = log_reg(X_train_oh, y_train.detect_threat, scaled_data)

print_report('Train', log_reg, scaled_data.transform(X_train_oh), y_train.detect_threat, log_reg.predict(scaled_data.transform(X_train_oh)))
print_report('Eval', log_reg, scaled_data.transform(X_eval_oh), y_eval.detect_threat, log_reg.predict(scaled_data.transform(X_eval_oh)), plot_pr=True)

rf = random_f(X_train_oh, y_train.detect_threat)

print_report('Train', rf, X_train_oh, y_train.detect_threat, rf.predict(X_train_oh))
print_report('Eval', rf, X_eval_oh, y_eval.detect_threat, rf.predict(X_eval_oh), plot_pr=True)





(16137183, 79)
Index(['bwd_psh_flags', 'bwd_urg_flags', 'fwd_byts_b_avg', 'fwd_pkts_b_avg',
       'fwd_blk_rate_avg', 'bwd_byts_b_avg', 'bwd_pkts_b_avg',
       'bwd_blk_rate_avg'],
      dtype='object')
[33, 35, 57, 58, 59, 60, 61, 62]
(16137183, 71)
Remaining amount of features: 31


/home/am520/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass strategy=most_frequent as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Classification Report (Train):


/home/am520/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/am520/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/am520/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.83      1.00      0.91   9373174
           1       0.00      0.00      0.00   1922854

    accuracy                           0.83  11296028
   macro avg       0.41      0.50      0.45  11296028
weighted avg       0.69      0.83      0.75  11296028

Avg Precision Score: 0.1702239052523595


Classification Report (Eval):


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 20.5min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   58.7s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  2.2min finished


Classification Report (Train):
              precision    recall  f1-score   support

           0       1.00      0.99      1.00   9373174
           1       0.97      0.98      0.98   1922854

    accuracy                           0.99  11296028
   macro avg       0.98      0.99      0.99  11296028
weighted avg       0.99      0.99      0.99  11296028

Avg Precision Score: 0.9562612441283196




[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   12.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   27.9s finished


Classification Report (Eval):
              precision    recall  f1-score   support

           0       0.99      0.99      0.99   2008537
           1       0.96      0.95      0.96    412040

    accuracy                           0.99   2420577
   macro avg       0.98      0.97      0.97   2420577
weighted avg       0.99      0.99      0.99   2420577

Avg Precision Score: 0.9258319487582557


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   12.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   28.4s finished
